In [2]:
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


In [4]:
!python --version

Python 3.8.9


###### 네이버에서 검색어를 입력받아 검색한 후 블로그 메뉴를 선택하고  <br>오른쪽 옵션버튼을 눌러서 정렬방식과 기간을 선택한다.

In [47]:
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By

import time
import tqdm
from tqdm.notebook import tqdm

In [48]:
query_text = input('어느지역 맛집을 찾으시나요?')
query_text

어느지역 맛집을 찾으시나요?구로디지털단지역 맛집


'구로디지털단지역 맛집'

In [49]:
path='C:\webdriver\chromedriver.exe'

options = webdriver.ChromeOptions()
options.add_argument('window_size=1600 , 900')
driver = webdriver.Chrome(path)

driver.get('https://www.naver.com')

C:\Users\LG\AppData\Local\Temp/ipykernel_7648/841033336.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [12]:
driver.find_element_by_link_text("카페").click()

C:\Users\LG\AppData\Local\Temp/ipykernel_7648/3740919148.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_link_text("카페").click()


In [19]:

driver.find_element(By.LINK_TEXT, '카페').click()
time.sleep(1)



In [50]:
#네이버 검색창에 검색어 입력
element = driver.find_element(By.ID, 'query')
element.send_keys(query_text)
element.submit()
time.sleep(1)

In [51]:
#'VIEW' 클릭
driver.find_element(By.LINK_TEXT, 'VIEW').click()

In [52]:
# '블로그' 클릭 (XPATH 활용)
driver.find_element(By.XPATH, '//*[@id="snb"]/div[1]/div/div[1]/a[2]').click()

In [53]:
#옵션 클릭
driver.find_element(By.XPATH, '//*[@id="snb"]/div[1]/div/div[2]/a').click()

# 정렬은 관련도순으로 기간은 6개월

driver.find_element(By.XPATH, '//*[@id="snb"]/div[2]/ul/li[2]/div/div/a[1]').click()

driver.find_element(By.XPATH, '//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[7]').click()


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=96.0.4664.45)
Stacktrace:
Backtrace:
	Ordinal0 [0x00646903+2517251]
	Ordinal0 [0x005DF8E1+2095329]
	Ordinal0 [0x004E2710+1058576]
	Ordinal0 [0x0050E324+1237796]
	Ordinal0 [0x00504037+1196087]
	Ordinal0 [0x005264D3+1336531]
	Ordinal0 [0x00503A36+1194550]
	Ordinal0 [0x005265BA+1336762]
	Ordinal0 [0x00535BBF+1399743]
	Ordinal0 [0x0052639B+1336219]
	Ordinal0 [0x005027A7+1189799]
	Ordinal0 [0x00503609+1193481]
	GetHandleVerifier [0x007D5904+1577972]
	GetHandleVerifier [0x00880B97+2279047]
	GetHandleVerifier [0x006D6D09+534521]
	GetHandleVerifier [0x006D5DB9+530601]
	Ordinal0 [0x005E4FF9+2117625]
	Ordinal0 [0x005E98A8+2136232]
	Ordinal0 [0x005E99E2+2136546]
	Ordinal0 [0x005F3541+2176321]
	BaseThreadInitThunk [0x7663FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77987A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77987A6E+238]


In [54]:
#스크롤 다운

def scroll_down(driver):
    driver.execute_script('window.scrollTo(0, 99999999)')
    time.sleep(1)
    
for i in range(10):
    scroll_down(driver)

In [55]:
url_list = []
title_list = []

#URL크롤링
articles = '.api_txt_lines.total_tit'
article_raw = driver.find_elements(By.CSS_SELECTOR , articles)


for article in article_raw:
#     print(article)
    url = article.get_attribute('href')
    url_list.append(url)
    
for article in article_raw:
    title = article.text
    title_list.append(title)
#     print(title)
print(' ')
print("url갯수 :" , len(url_list))
print("title갯수 :" , len(title_list))

 
url갯수 : 330
title갯수 : 330


In [62]:
#판다스를 활용한 데이터 프레임 만들고 excel에 저장

df = pd.DataFrame({'url': url_list , 'title':title_list})

#저장하기
df.to_excel("blog_url.xlsx",'w',encoding='UTF-8-SIG')

###### 블로그 내용 크롤링하기

In [65]:
#csv or xlsx 파일을 불러오기

url_load = pd.read_excel("blog_url.xlsx")
url_load

,Unnamed: 0,url,title
0,0,https://blog.naver.com/ssj7748?Redirect=Log&lo...,으뜸이던 구로디지털단지역 맛집
1,1,https://blog.naver.com/kimmso1?Redirect=Log&lo...,빈틈없는 구로디지털단지역 맛집 모음
2,2,https://blog.naver.com/80ggongju?Redirect=Log&...,옹골찼던 구로디지털단지역 맛집 리스트
3,3,https://blog.naver.com/flyfree6?Redirect=Log&l...,공감했던 구로디지털단지역 맛집 소개
4,4,https://blog.naver.com/bal2000?Redirect=Log&lo...,상당했던 구로디지털단지역 맛집 리스트
...,...,...,...
325,325,https://blog.naver.com/yiji6274?Redirect=Log&l...,구로디지털단지역 점심식사 사조참치 알탕 대구지리 실하다
326,326,https://blog.naver.com/icandoitonce?Redirect=L...,[구로디지털단지][다케롤]가성비 좋은 단골 참치집
327,327,https://blog.naver.com/torarei?Redirect=Log&lo...,"[구로디지털단지역, 구로동] 형제특수부위 껍데기편"
328,328,https://blog.naver.com/abwlr22?Redirect=Log&lo...,[구로디지털] - 내가 먹어본 뼈해장국중에 살이 제일 많은 집...


In [ ]:
dict = {}

#수집할 글이 갯수
number = 30
url_load['url']
for i in tqdm(range(number)):
    url = url_load['url'][i]
    driver = webdriver.Chrome(path)
    driver.get(url)
    try:
        #iframe 접근
        driver.switch_to.frame('mainFrame')
        
        target_info = {}
        
        #제목 크롤링 
        overlays = ".se-module.se-module-text.se-title-text"
        title = driver.find_element(By.CSS_SELECTOR, overlays)
        title = title.text
#         print(title)
        #글쓴이 크롤링 
        overlays = ".nick"
        nick = driver.find_element(By.CSS_SELECTOR, overlays)
        nick = nick.text
#         print(nick)
        #날짜 크롤링 
        overlays = ".se_publishDate.pcol2"
        date = driver.find_element(By.CSS_SELECTOR, overlays)
        date = date.text
#         print(date)
        #내용 크롤링 
        overlays = ".se-component.se-text.se-l-default"
        contents = driver.find_elements(By.CSS_SELECTOR, overlays)
#         print(contents.text)
        content_list= []
        for content in contents:
#             print('test4')
            content_list.append(content.text)
        content_str = " ".join(content_list)
        
        #target_info 에 추가
        target_info['title'] = title
        target_info['nickname'] = nick
        target_info['data'] = date
        target_info['content'] = content_str
        
        dict[i] = target_info
        driver.close()
    except:
        driver.close()
        print('test')
        time.sleep(1)
        continue
        
    #
    
print(dict)  

  0%|          | 0/30 [00:00<?, ?it/s]

C:\Users\LG\AppData\Local\Temp/ipykernel_7648/1683603413.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
